## Advanced Modeling

### Author: Tilova Shahrin

- [Reading Dataframe](#df)
- [Test and Train Split](#split)
- [Logreg Scoring](#logreg)
- [Multi-Output Classifier](#multi)
- [Multi-Output Classifier With Different Weights](#multiweights)
- [Randomized Search CV](#search)
- [Prediction Report](#pred)
- [References](#ref)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
import warnings
warnings.filterwarnings('ignore')

<a id='df'><a/>
### Reading Dataframe

In [3]:
parking_df = pd.read_csv('../data/parking_df.csv')

In [55]:
parking_df.head()

,datetime_of_infraction,time_of_infraction,year,month,day,infraction_code,infraction_description,set_fine_amount,location2,province,...,longitude,permit_time_restrictions,fee_related,time_related,plate_related,fire_route,accessible_related,commercial_related,obstruction_related,cycle_related
0,2016-12-30 16:37:00,16:37:00,2016,12,30,403.0,STOP-SIGNED HIGHWAY-RUSH HOUR,150,"1546 BLOOR ST W, TORONTO, ON, CANADA",ON,...,-79.453142,0,0,1,0,0,0,0,0,0
1,2016-12-30 16:37:00,16:37:00,2016,12,30,403.0,STOP-SIGNED HIGHWAY-RUSH HOUR,150,"5418 YONGE ST, TORONTO, ON, CANADA",ON,...,-79.414671,0,0,1,0,0,0,0,0,0
2,2016-12-30 16:37:00,16:37:00,2016,12,30,403.0,STOP-SIGNED HIGHWAY-RUSH HOUR,150,"777 QUEEN ST W, TORONTO, ON, CANADA",ON,...,-79.408080,0,0,1,0,0,0,0,0,0
3,2016-12-30 16:37:00,16:37:00,2016,12,30,403.0,STOP-SIGNED HIGHWAY-RUSH HOUR,150,"747 QUEEN ST E, TORONTO, ON, CANADA",ON,...,-79.348080,0,0,1,0,0,0,0,0,0
4,2016-12-30 16:37:00,16:37:00,2016,12,30,403.0,STOP-SIGNED HIGHWAY-RUSH HOUR,150,"3042 DUNDAS ST W, TORONTO, ON, CANADA",ON,...,-79.470785,0,0,1,0,0,0,0,0,0


In [4]:
parking_coord = parking_df[parking_df['latitude'] != 0.0]

<a id='split'><a/>
### Test Train Split

In [5]:
df_numerical_copy = parking_coord[['latitude', 'longitude', 'permit_time_restrictions', 'fee_related', 'time_related', 'fire_route', 'accessible_related', 'commercial_related', 'obstruction_related', 'cycle_related']]

X = df_numerical_copy[['latitude', 'longitude']]
y = df_numerical_copy.drop(columns=['latitude', 'longitude'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
y.columns

Index(['permit_time_restrictions', 'fee_related', 'time_related', 'fire_route',
       'accessible_related', 'commercial_related', 'obstruction_related',
       'cycle_related'],
      dtype='object')

<a id='logreg'><a/>
### Logistic Regression, predicts type of infraction based on location

In [9]:
models = {}

for column in y.columns:
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = LogisticRegression()
    model.fit(X_train, y_train[column])
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test[column], y_pred)
    
    models[column] = {'model': model, 'accuracy': accuracy}

for infraction, model_info in models.items():
    print(f"Infraction: {infraction}, Accuracy: {model_info['accuracy']}")


Infraction: permit_time_restrictions, Accuracy: 0.5955489654451519
Infraction: fee_related, Accuracy: 0.781054551015391
Infraction: time_related, Accuracy: 0.7142270890961848
Infraction: fire_route, Accuracy: 0.9779964730864752
Infraction: accessible_related, Accuracy: 0.9881319940611577
Infraction: commercial_related, Accuracy: 0.9838036852568434
Infraction: obstruction_related, Accuracy: 0.9931049034163829
Infraction: cycle_related, Accuracy: 0.9925745113714893


<a id='multi'><a/>
### Multi-Output Classifier

To make sure we output all features, instead of a for loop, we can use a multi-output classifier. This will show the score for all my infractions. 

In [58]:
classifier = MultiOutputClassifier(RandomForestClassifier(random_state=42))

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.81      0.82    241602
           1       0.69      0.58      0.63    113107
           2       0.75      0.61      0.67    144335
           3       0.73      0.15      0.25     11367
           4       0.65      0.48      0.56      6754
           5       0.69      0.47      0.56      8367
           6       0.63      0.24      0.34     12675
           7       0.62      0.02      0.05      3836

   micro avg       0.77      0.67      0.71    542043
   macro avg       0.70      0.42      0.48    542043
weighted avg       0.76      0.67      0.70    542043
 samples avg       0.62      0.62      0.62    542043



For obstruction related,commercial, accesible and cycle related infractions, there is a large data imbalance. 

With these scores, it's important to note that we should consider the context of the problem and the relative importance of precision and recall. In this case, the number of difference offences has a disadvantage in proportion. Let's try to upsample some of these models to improve their scores.

To balance these targets,let's use the `class_weight` parameter in random forest. 
Refer to: https://stackoverflow.com/questions/58275113/proper-use-of-class-weight-parameter-in-random-forest-classifier

Let's try two methods, one using `"balanced"` and another as a dictionary.

<a id='multiweights'><a/>
### Multi-Output Classifier with different weight class

In [59]:
classifier = MultiOutputClassifier(RandomForestClassifier(random_state=42, class_weight='balanced'))

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82    241602
           1       0.51      0.84      0.64    113107
           2       0.61      0.80      0.69    144335
           3       0.10      0.86      0.17     11367
           4       0.17      0.93      0.29      6754
           5       0.25      0.98      0.39      8367
           6       0.09      0.81      0.16     12675
           7       0.08      0.96      0.15      3836

   micro avg       0.47      0.82      0.60    542043
   macro avg       0.33      0.87      0.41    542043
weighted avg       0.64      0.82      0.70    542043
 samples avg       0.55      0.78      0.62    542043



Based on both this Random Forest Classifier, it seems the scores are not that much different compared to the model with balanced weighted class. 

In [6]:
#create dictionary for class_weight
weight = {0: 2, 1: 5, 2: 5, 3: 100, 4: 100, 5: 100, 6: 200, 7: 200}
classifier = MultiOutputClassifier(RandomForestClassifier(random_state=42, class_weight = weight))

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.92      0.81    241602
           1       0.58      0.75      0.66    113107
           2       0.61      0.80      0.69    144335
           3       0.48      0.30      0.37     11367
           4       0.60      0.54      0.57      6754
           5       0.54      0.83      0.65      8367
           6       0.57      0.27      0.37     12675
           7       0.42      0.28      0.34      3836

   micro avg       0.65      0.81      0.72    542043
   macro avg       0.56      0.59      0.56    542043
weighted avg       0.65      0.81      0.72    542043
 samples avg       0.68      0.77      0.71    542043



These scores are much better, only obstruction and cycle related infractions are of concern. 

<a id='search'><a/>
### Grid Search

#### Pipeline for RandomForestClassifier

In [10]:
from sklearn.pipeline import Pipeline
rf = RandomForestClassifier(random_state=42)
pipe = Pipeline(steps=[('clf', MultiOutputClassifier(rf))])
param_grid = {
    'clf__estimator__n_estimators': [300],  #Number of trees in the forest
    'clf__estimator__max_depth': [15, 30],  #Maximum depth of the trees
}


#fit
pipe.fit(X_train, y_train)

#score
pipe.score(X_test, y_test)

0.6157832283841045

In [11]:
from sklearn.model_selection import RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_grid,
    scoring='accuracy',
    refit='precision_micro',
    cv = 3,
    verbose= 2
)

In [12]:
fittedgrid = search.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END clf__estimator__max_depth=15, clf__estimator__n_estimators=300; total time=22.8min
[CV] END clf__estimator__max_depth=15, clf__estimator__n_estimators=300; total time=23.0min
[CV] END clf__estimator__max_depth=15, clf__estimator__n_estimators=300; total time=23.0min
[CV] END clf__estimator__max_depth=30, clf__estimator__n_estimators=300; total time=23.8min
[CV] END clf__estimator__max_depth=30, clf__estimator__n_estimators=300; total time=23.5min
[CV] END clf__estimator__max_depth=30, clf__estimator__n_estimators=300; total time=23.6min


In [13]:
fittedgrid.score(X_train, y_train)

0.6187820278298353

In [14]:
fittedgrid.score(X_test, y_test)

0.6177944595324419

In [15]:
fittedgrid.best_estimator_

Pipeline(steps=[('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=30,
                                                                        n_estimators=300,
                                                                        random_state=42)))])

In [16]:
fittedgrid.best_params_

{'clf__estimator__n_estimators': 300, 'clf__estimator__max_depth': 30}

In [21]:
test_score = pipe.score(X_test, y_test)
print("Test Accuracy:", test_score)

Test Accuracy: 0.6157832283841045


<a id=''></a>
### Prediction Report

Checking the predictions using a classification report. 

In [22]:
y_pred = fittedgrid.best_estimator_.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.81      0.82    241602
           1       0.69      0.58      0.63    113107
           2       0.75      0.61      0.67    144335
           3       0.73      0.15      0.25     11367
           4       0.65      0.48      0.56      6754
           5       0.69      0.47      0.56      8367
           6       0.63      0.24      0.34     12675
           7       0.62      0.02      0.05      3836

   micro avg       0.77      0.67      0.72    542043
   macro avg       0.70      0.42      0.48    542043
weighted avg       0.76      0.67      0.70    542043
 samples avg       0.62      0.62      0.62    542043



It's important to note that the reason for the low score is from the class imbalance of number 3, 6, and 7. Which are `time_related`, `commercial_related`, `cycle_related`.

### Prediction Testing

In [24]:
X_train

,latitude,longitude
1180920,43.629106,-79.475411
4512486,43.704912,-79.388689
11521333,43.641681,-79.383041
2707075,43.648442,-79.398237
8580979,43.764867,-79.383870
...,...,...
1699382,43.655527,-79.385242
10329201,43.650837,-79.386621
835327,43.775587,-79.414671
4473564,43.655527,-79.385242


In [27]:
my_predictions

array([[0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0]])

Let's apply this as a demo situation, and code the output of this prediction.

In [88]:
input_pred = np.array([[43.629106, -79.475411]])
my_predictions = fittedgrid.best_estimator_.predict(input_pred)

indices = np.where(my_predictions == 1)
fine_amounts = parking_df.iloc[indices[0]]['set_fine_amount']
labels = y.columns[indices[1]]

# Print the fine amounts and labels
for fine_amount, label in zip(fine_amounts, labels):
    print(f"This model predicts '{label}' parking tickets for this location with a fine of ${fine_amount}.")

This model predicts 'fee_related' parking tickets for this location with a fine of $150.


Finally, let's save this model.

In [20]:
import joblib 

# Save the model as a pickle in a file 
joblib.dump(fittedgrid, 'model_custom_best.pkl') 

['model_custom_best.pkl']

### References

- Class Weight: https://stackoverflow.com/questions/58275113/proper-use-of-class-weight-parameter-in-random-forest-classifier
- Multi-Output Classifier: https://calmcode.io/course/scikit-meta/multi-output#:~:text=Using%20the%20MultiOutputClassifier&text=Instead%20of%20making%20two%20pipelines,this%20by%20using%20a%20MultiOutputClassifier.
- Tuning params for multiclassification output: https://datascience.stackexchange.com/questions/107867/how-to-train-multioutput-classification-with-hyperparameter-tuning-in-sklearn
- Metrics for scoring parameters: https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules
- Random Forest Classifier: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
- Class Weight for Random Forest Classifier: https://stackoverflow.com/questions/58275113/proper-use-of-class-weight-parameter-in-random-forest-classifier
- RandomizedSearchCV: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

### Source 

https://open.toronto.ca/dataset/parking-tickets/